# 03 - Model Execution & Evaluation
##### In this notebook, we:
##### 1. Load and transform the data
##### 2. Train tree based Model
##### 3. Implement AutoML based approach for Mass Model Training
##### 4. Evaluate results and give model insight

## 1. Loading Data

Loading master table into model pipeline

In [3]:
cmc_model = spark.table("cmc_db.model_data_csv")

cmc_model.cache()

#display(cmc_model.select("*"))

Out[12]: DataFrame[StationName: string, Latitude: string, Longitude: string, GroupCode: string, Time: string, month: string, year: string, HUC12: string, do: string, tn: string, tp: string, watertemp: string, airtemp_narr: string, precip3_narr: string, precip24_narr: string, precip48_narr: string, windspeed_narr
: string]

## Transforming Data

Applying transformations to generalize data

In [5]:
from pyspark.sql.functions import *

names = cmc_model.schema.names

for name in names:
  cmc_model = cmc_model.withColumn(name, regexp_replace(name, 'nan', ''))
  cmc_model = cmc_model.withColumn(name, regexp_replace(name, 'None', ''))
  cmc_model = cmc_model.withColumn(name, regexp_replace(name, 'mm', ''))
  cmc_model = cmc_model.withColumn(name, regexp_replace(name, 'Knots', ''))
  cmc_model = cmc_model.withColumn(name, regexp_replace(name, 'knots', ''))
  cmc_model = cmc_model.withColumn(name, regexp_replace(name, '\\"\"\""', ''))
  cmc_model = cmc_model.withColumn(name, regexp_replace(name, '>', ''))

#display(cmc_model)

In [6]:
from pyspark.sql.types import IntegerType, TimestampType, DoubleType, StringType, DecimalType, FloatType

cmc_model_schema = cmc_model.withColumn("time", cmc_model["time"].cast(StringType()))
cmc_model_schema = cmc_model_schema.withColumn("Latitude", cmc_model_schema["Latitude"].cast(DoubleType()))
cmc_model_schema = cmc_model_schema.withColumn("Longitude", cmc_model_schema["Longitude"].cast(DoubleType()))
cmc_model_schema = cmc_model_schema.withColumn("month", cmc_model_schema["month"].cast(FloatType()))
cmc_model_schema = cmc_model_schema.withColumn("year", cmc_model_schema["year"].cast(FloatType()))
cmc_model_schema = cmc_model_schema.withColumn("do", cmc_model_schema["do"].cast(DoubleType()))
cmc_model_schema = cmc_model_schema.withColumn("tn", cmc_model_schema["tn"].cast(DoubleType()))
cmc_model_schema = cmc_model_schema.withColumn("tp", cmc_model_schema["tp"].cast(DoubleType()))
cmc_model_schema = cmc_model_schema.withColumn("watertemp", cmc_model_schema["watertemp"].cast(DoubleType()))
cmc_model_schema = cmc_model_schema.withColumn("airtemp_narr", cmc_model_schema["airtemp_narr"].cast(DoubleType()))
cmc_model_schema = cmc_model_schema.withColumn("precip3_narr", cmc_model_schema["precip3_narr"].cast(DoubleType()))
cmc_model_schema = cmc_model_schema.withColumn("precip24_narr", cmc_model_schema["precip24_narr"].cast(DoubleType()))
cmc_model_schema = cmc_model_schema.withColumn("precip48_narr", cmc_model_schema["precip48_narr"].cast(DoubleType()))
cmc_model_schema = cmc_model_schema.withColumn("windspeed_narr\r", cmc_model_schema["windspeed_narr\r"].cast(DoubleType()))
cmc_model_schema = cmc_model_schema.withColumn("HUC12", cmc_model_schema["HUC12"].cast(DoubleType()))
cmc_model_schema = cmc_model_schema.withColumn("StationName", cmc_model_schema["StationName"].cast(StringType()))
cmc_model_schema = cmc_model_schema.withColumn("GroupCode", cmc_model_schema["GroupCode"].cast(StringType()))
cmc_model_schema = cmc_model_schema.withColumnRenamed("windspeed_narr\r","windspeed_narr")

#display(cmc_model_schema)

In [7]:
from pyspark.sql.functions import lower, col

cmc_model_schema_rep = cmc_model_schema.replace('', 'None')
cmc_model_schema_rep = cmc_model_schema_rep.fillna('') # needs to eliminate nulls

names = ["StationName", "GroupCode", "time"]

for name in names:
  cmc_model_schema_rep = cmc_model_schema_rep.withColumn(name,lower(col(name)))

#display(cmc_model_schema_rep)

In [8]:
cmc_model_schema_rep.printSchema()

root
-- StationName: string (nullable = false)
-- Latitude: double (nullable = true)
-- Longitude: double (nullable = true)
-- GroupCode: string (nullable = false)
-- time: string (nullable = false)
-- month: float (nullable = true)
-- year: float (nullable = true)
-- HUC12: double (nullable = true)
-- do: double (nullable = true)
-- tn: double (nullable = true)
-- tp: double (nullable = true)
-- watertemp: double (nullable = true)
-- airtemp_narr: double (nullable = true)
-- precip3_narr: double (nullable = true)
-- precip24_narr: double (nullable = true)
-- precip48_narr: double (nullable = true)
-- windspeed_narr: double (nullable = true)

## StringIndexer

Transform categorical strings with StringIndexer

In [10]:
from pyspark.ml.feature import StringIndexer

iStationName = StringIndexer(inputCol="StationName", outputCol="cat_StationName", handleInvalid="skip")
iGroupCode = StringIndexer(inputCol="GroupCode", outputCol="cat_GroupCode", handleInvalid="skip")
iTime = StringIndexer(inputCol="time", outputCol="cat_Time", handleInvalid="skip")

## Imputer

Impute possible missing double values

In [12]:
from pyspark.ml.feature import Imputer

imputer = Imputer()
imputer.setStrategy("median")

imputeCols = ["Latitude",
  "Longitude",
  "month",
  "year",
  "HUC12",
  "do",
  "tn",
  "tp",
  "watertemp",
  "airtemp_narr",
  "precip3_narr",
  "precip24_narr",
  "precip48_narr",
  "windspeed_narr"]
imputer.setInputCols(imputeCols)
imputer.setOutputCols(imputeCols)

cmc_model_schema_rep = imputer.fit(cmc_model_schema_rep).transform(cmc_model_schema_rep)

#display(cmc_model_schema_rep)

In [13]:
recordCount = cmc_model_schema_rep.count()
noNullsRecordCount = cmc_model_schema_rep.na.drop().count()

print("We have {} records that contain null values.".format(recordCount - noNullsRecordCount))

We have 0 records that contain null values.

## OneHotEncoder

One hot encoding of indexed strings

In [15]:
from pyspark.ml.feature import OneHotEncoderEstimator

oneHotEnc = OneHotEncoderEstimator()
oneHotEnc.setInputCols(["cat_StationName", "cat_GroupCode", "cat_Time"])
oneHotEnc.setOutputCols(["vec_StationName", "vec_GroupCode", "vec_Time"])

Out[20]: OneHotEncoderEstimator_f65dd78e0d4e

## 2. Train/Test Split

80/20 split for model train and test

In [17]:
seed = 1234

testDF, trainDF = cmc_model_schema_rep.randomSplit((0.2, 0.8), seed=seed)

print(testDF.count(), trainDF.count())

display(testDF)

StationName,Latitude,Longitude,GroupCode,time,month,year,HUC12,do,tn,tp,watertemp,airtemp_narr,precip3_narr,precip24_narr,precip48_narr,windspeed_narr
8-col-40-laca,37.984722,-77.766944,laca,10:10:00,8.0,2017.0,2.0801060603E10,7.87,0.34,0.01,29.45,24.526764,1.4765625,0.59713733,12.6953125,2.9420943
8-col-40-laca,37.984722,-77.766944,laca,10:20:00,6.0,2013.0,2.0801060603E10,7.14,0.4,0.01,28.28,24.393707,0.0625,0.31296083,8.664523,3.4805393
8-col-40-laca,37.984722,-77.766944,laca,9:00:00,8.0,2018.0,2.0801060603E10,8.09,0.44,0.02,31.79,22.972809,0.25258133,2.0625,4.6683507,9.122026
8-col-40-laca,37.984722,-77.766944,laca,9:00:00,10.0,2018.0,2.0801060603E10,8.5,0.48,0.02,25.67,19.233307,0.0,0.0,0.03125,13.361147
8-col-40-laca,37.984722,-77.766944,laca,9:16:00,8.0,2019.0,2.0801060603E10,7.76,0.38,0.02,29.03,24.447418,0.1640625,0.0,14.3828125,20.511784
8-col-40-laca,37.984722,-77.766944,laca,9:18:00,4.0,2019.0,2.0801060603E10,9.79,0.44,0.05,18.71,18.951813,0.0,5.7578125,5.7578125,5.4579177
8-col-40-laca,37.984722,-77.766944,laca,9:45:00,4.0,2016.0,2.0801060603E10,8.97,0.48,0.02,15.54,-0.4128723,0.0,1.6875,1.6875,95.04044
8-col-40-laca,37.984722,-77.766944,laca,9:50:00,8.0,2016.0,2.0801060603E10,6.86,0.38,0.01,32.8,24.508453,0.0,0.09611061,3.414179,2.6908665
8-crc-44-laca,38.088111,-77.901194,laca,9:00:00,10.0,2019.0,2.0801060404E10,8.32,0.64,0.06,21.68,15.24057,0.109375,0.1875,0.59375,57.71276
8-crc-44-laca,38.088111,-77.901194,laca,9:45:00,8.0,2019.0,2.0801060404E10,6.34,0.68,0.07,28.3,24.447418,0.1640625,0.0,14.3828125,20.511784


In [18]:
featureCols = [
  "Latitude",
  "Longitude",
  "month",
  "year",
  "HUC12",
  "do",
  "tp",
  "watertemp",
  "airtemp_narr",
  "precip3_narr",
  "precip24_narr",
  "precip48_narr",
  "windspeed_narr",
  "vec_StationName", 
  "vec_GroupCode", 
  "vec_Time"
]

## Baseline Model Linear Regression

Creating baseline model to extablish metrics for other models to beat

In [20]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

vectorAssembler = VectorAssembler(inputCols=featureCols, outputCol="features")

lr = (LinearRegression()
      .setLabelCol("tn")
      .setFeaturesCol("features"))

lrpipeline = Pipeline(stages = [iStationName, iGroupCode, iTime, oneHotEnc, vectorAssembler, lr])

lrpipeline.getStages()

lrPipelineModel = lrpipeline.fit(trainDF)

predictedDF_lr = lrPipelineModel.transform(testDF)

In [21]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.evaluation import RegressionMetrics

def printEval(df):
  out = df\
  .select("prediction", "tn").rdd.map(lambda x: (float(x[0]), float(x[1])))
  
  metrics = RegressionMetrics(out)

  mse = str(metrics.meanSquaredError)
  rmse = str(metrics.rootMeanSquaredError)
  r2 = str(metrics.r2)
  mae = str(metrics.meanAbsoluteError)
  ve = str(metrics.explainedVariance)

  print("RMSE: {}\nR2: {}\nMSE: {}\nMAE: {}\nVariance Explained: {}".format(rmse, r2, mse, mae, ve))

In [22]:
# Metrics to beat

printEval(predictedDF_lr)

RMSE: 1.3352019655186673
R2: 0.33305892314168517
MSE: 1.7827642887249122
MAE: 0.9148119756791776
Variance Explained: 1.037181236656476

## Random Forrest

In [24]:
from pyspark.ml.regression import RandomForestRegressor

rf = RandomForestRegressor(labelCol="tn", 
                           featuresCol="features",
                           maxDepth=15, 
                           maxBins=10, 
                           minInstancesPerNode=1, 
                           minInfoGain=0.0, 
                           maxMemoryInMB=256, 
                           cacheNodeIds=False, 
                           checkpointInterval=10, 
                           subsamplingRate=1.0, 
                           seed=None, 
                           numTrees=500, 
                           featureSubsetStrategy='auto'
                          )

rfpipeline = Pipeline(stages = [iStationName, iGroupCode, iTime, oneHotEnc, vectorAssembler, rf])

rfPipelineModel = rfpipeline.fit(trainDF)

predictedDF_rf = rfPipelineModel.transform(testDF)

In [25]:
printEval(predictedDF_rf)

RMSE: 0.7733119914935228
R2: 0.7762809173668949
MSE: 0.5980114361876783
MAE: 0.4493976915351382
Variance Explained: 1.8882213265937544

## GBT

In [27]:
from pyspark.ml.regression import GBTRegressor

gbt = GBTRegressor(labelCol="tn", 
                   featuresCol="features", 
                   maxIter=10)

gbtpipeline = Pipeline(stages = [iStationName, iGroupCode, iTime, oneHotEnc, vectorAssembler, gbt])

gbtPipelineModel = gbtpipeline.fit(trainDF)

predictedDF_gbt = gbtPipelineModel.transform(testDF)

In [28]:
printEval(predictedDF_gbt)

RMSE: 0.9569552642023226
R2: 0.6574083196959442
MSE: 0.915763377684537
MAE: 0.637145980034372
Variance Explained: 1.7340869158589196

## XGBoost

Please install xgboost and numpy!

In [30]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, explained_variance_score, mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd
import math

seed = 1234

pipeline = Pipeline(stages=[iStationName, iGroupCode, iTime, oneHotEnc])

pipelineModel = pipeline.fit(cmc_model_schema_rep)
trainingData = pipelineModel.transform(cmc_model_schema_rep)

xgbfeatureCols = ["Latitude",
  "Longitude",
  "month",
  "year",
  "HUC12",
  "do",
  "tp",
  "watertemp",
  "airtemp_narr",
  "precip3_narr",
  "precip24_narr",
  "precip48_narr",
  "windspeed_narr",
  "tn"
   ]

xgbInputTrainingDF = trainingData.select(xgbfeatureCols)

xgbtestDF, xgbtrainDF = xgbInputTrainingDF.randomSplit((0.2, 0.8), seed=seed)
print(xgbtestDF.count(), xgbtrainDF.count())

#change to pandas for the xgb
pandas_xgbtrainDF = xgbtrainDF.toPandas()
pandas_xgbtestDF = xgbtestDF.toPandas()

6096 24225

In [31]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
                          alpha = 0.0, 
                          min_child_weight = 1.0,
                          tweedie_variance_power = 1.5,
                          random_state = 1234,
                          max_bin = 256,
                          gamma = 0.01,
                          max_depth = 10, 
                          colsample_bylevel = 1.0,
                          scale_pos_weight = 1,       
                          max_delta_step = 0.0, 
                          learning_rate = 1, 
                          n_estimators = 2000,
                          n_jobs = 1,
                          tree_method = 'auto', 
                          num_parallel_tree = 1,
                          colsample_bytree = 0.5,
                          subsample = 1.0, 
                          reg_lambda = 1.0)

xg_reg.fit(pandas_xgbtrainDF[pandas_xgbtrainDF.columns[:-1]], pandas_xgbtrainDF['tn'])

y_pred = xg_reg.predict(pandas_xgbtestDF[pandas_xgbtestDF.columns[:-1]])
y_true = pd.DataFrame({'actuals': pandas_xgbtestDF['tn'] })
y_pred = pd.DataFrame({'prediction': np.array(y_pred) })

[03:51:47] WARNING: /workspace/src/learner.cc:480: 
Parameters: { tweedie_variance_power } might not be used.

 This may not be accurate due to some parameters are only used in language bindings but
 passed down to XGBoost core. Or some parameters are not used but slip through this
 verification. Please open an issue if you find above cases.

In [32]:
r2 = r2_score(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)
ve = explained_variance_score(y_true, y_pred)

print("RMSE: {}\nR2: {}\nMSE: {}\nMAE: {}\nVariance Explained: {}".format(r2, rmse, mse, mae, ve))

RMSE: 0.7767041723450037
R2: 0.7955393090744154
MSE: 0.6328827922825981
MAE: 0.45335846370995736
Variance Explained: 0.7767049138628799

## 3. H2O AutoML

Pysparkling is not installed to the Databricks environment by default. You can install additional libraries very easily. Go ahead and install h2o_pysparkling_2.4, pysparkling, colorama==0.3.8.

In [34]:
from pysparkling import *
from pyspark.sql import SparkSession
import h2o
import xgboost

hc = H2OContext.getOrCreate(spark)

In [35]:
# Using non Spark Version

import h2o
from h2o.automl import H2OAutoML

folds = 5

aml = H2OAutoML(max_models = 60,
                max_runtime_secs = 5000,
                max_runtime_secs_per_model = 120,
                nfolds = folds,
                seed = 1234)

response = "tn"
predictor = ["Latitude",
  "Longitude",
  "month",
  "year",
  "HUC12",
  "do",
  "tp",
  "watertemp",
  "airtemp_narr",
  "precip3_narr",
  "precip24_narr",
  "precip48_narr",
  "windspeed_narr",
  "StationName", 
  "GroupCode", 
  "time"]

training_frame = h2o.H2OFrame(trainDF.toPandas())

aml.train(x = predictor, y = response, training_frame = training_frame)

Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%

In [36]:
leaderboard = h2o.automl.get_leaderboard(aml, extra_columns = 'ALL')
aml_leaderboard_df=leaderboard.as_data_frame()
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle,training_time_ms,predict_time_per_row_ms
0,StackedEnsemble_BestOfFamily_AutoML_20200612_0...,0.478472,0.691717,0.478472,0.299250,0.140713,1935,0.103678
1,XGBoost_grid__1_AutoML_20200612_004216_model_13,0.501005,0.707817,0.501005,0.316772,NaN,11804,0.016899
2,GBM_grid__1_AutoML_20200612_004216_model_4,0.503178,0.709350,0.503178,0.320131,0.149007,5850,0.091005
3,StackedEnsemble_AllModels_AutoML_20200612_004216,0.507509,0.712397,0.507509,0.295363,0.139292,4535,0.310028
4,XGBoost_grid__1_AutoML_20200612_004216_model_3,0.508372,0.713002,0.508372,0.321309,0.150678,18387,0.018961
5,XGBoost_grid__1_AutoML_20200612_004216_model_19,0.514605,0.717360,0.514605,0.321414,0.150402,14694,0.015378
6,XGBoost_grid__1_AutoML_20200612_004216_model_11,0.515831,0.718214,0.515831,0.329205,NaN,9121,0.008341
7,GBM_grid__1_AutoML_20200612_004216_model_9,0.515835,0.718216,0.515835,0.320651,0.149351,6653,0.068468
8,XGBoost_grid__1_AutoML_20200612_004216_model_9,0.522853,0.723086,0.522853,0.330457,0.152388,15562,0.012913
9,GBM_3_AutoML_20200612_004216,0.524458,0.724195,0.524458,0.343013,NaN,5342,0.105492


In [37]:
for m in aml.leaderboard.as_data_frame()['model_id']:
    print(m)
    print(h2o.get_model(m))

StackedEnsemble_BestOfFamily_AutoML_20200612_004216
Model Details
=============
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_BestOfFamily_AutoML_20200612_004216

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.24617756721917858
RMSE: 0.49616284344878003
MAE: 0.14920699545240343
RMSLE: 0.07349810890166997
R^2: 0.9268891443645427
Mean Residual Deviance: 0.24617756721917858
Null degrees of freedom: 10021
Residual degrees of freedom: 10019
Null deviance: 33745.94852217975
Residual deviance: 2467.1915786706077
AIC: 14401.344656764826

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.47847185063434794
RMSE: 0.6917165970499392
MAE: 0.2992496882248374
RMSLE: 0.14071251719192907
R^2: 0.8434131329089226
Mean Residual Deviance: 0.47847185063434794
Null degrees of freedom: 24224
Residual degrees of freedom: 24222
Null deviance: 74025.41236472882
Residual deviance: 11590.98058161708
AIC: 50897.92185059927

XGBoost_grid__1_AutoML_20200612_004216_model_13
Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_grid__1_AutoML_20200612_004216_model_13


Model Summary: 
 number_of_trees
-- -----------------
 51


ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.08432889058822349
RMSE: 0.29039437079293307
MAE: 0.11705731425061437
RMSLE: 0.060013769950949904
Mean Residual Deviance: 0.08432889058822349

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 0.5010054231970625
RMSE: 0.7078173657074701
MAE: 0.316772349769609
RMSLE: NaN
Mean Residual Deviance: 0.5010054231970625

Cross-Validation Metrics Summary: 
 mean sd cv_1_valid cv_2_valid cv_3_valid cv_4_valid cv_5_valid
---------------------- -------- ---------- ------------ ------------ ------------ ------------ ------------
mae 0.316772 0.00317831 0.319462 0.315005 0.313057 0.320654 0.315684
mean_residual_deviance 0.501005 0.274601 0.477308 0.350885 0.981371 0.327879 0.367584
mse 0.501005 0.274601 0.477308 0.350885 0.981371 0.327879 0.367584
r2 0.842041 0.0608288 0.838828 0.871324 0.738029 0.887206 0.874819
residual_deviance 0.501005 0.274601 0.477308 0.350885 0.981371 0.327879 0.367584
rmse 0.690553 0.173715 0.690875 0.592355 0.990642 0.572607 0.606287
rmsle 0.150458 0.00358092 nan 0.151691 0.146189 0.149319 0.154633

Scoring History: 
 timestamp duration number_of_trees training_rmse training_mae training_deviance
-- ------------------- ----------------- ----------------- --------------- -------------- -------------------
 2020-06-12 01:08:21 14 min 53.100 sec 0 2.22269 1.4136 4.94037
 2020-06-12 01:08:22 14 min 53.729 sec 5 0.845643 0.396743 0.715111
 2020-06-12 01:08:23 14 min 54.410 sec 10 0.608686 0.248764 0.370498
 2020-06-12 01:08:24 14 min 55.376 sec 15 0.515188 0.201671 0.265418
 2020-06-12 01:08:24 14 min 56.286 sec 20 0.460346 0.177569 0.211918
 2020-06-12 01:08:25 14 min 57.246 sec 25 0.418055 0.162916 0.17477
 2020-06-12 01:08:26 14 min 58.335 sec 30 0.387147 0.150626 0.149883
 2020-06-12 01:08:28 14 min 59.456 sec 35 0.358197 0.139768 0.128305
 2020-06-12 01:08:29 15 min 0.621 sec 40 0.335482 0.133688 0.112549
 2020-06-12 01:08:30 15 min 1.842 sec 45 0.315221 0.126582 0.0993645
 2020-06-12 01:08:31 15 min 3.143 sec 50 0.295465 0.119407 0.0872993
 2020-06-12 01:08:32 15 min 4.083 sec 51 0.290394 0.117057 0.0843289

Variable Importances: 
variable relative_importance scaled_importance percentage
------------------------- --------------------- --------------------- ----------------------
Longitude 46752.0859375 1.0 0.36528417218927267
Latitude 31911.30078125 0.6825642137959419 0.24932990380247236
HUC12 13435.775390625 0.2873834422829062 0.104976622815215
tp 9104.6708984375 0.1947436294202782 0.07113676546192081
GroupCode.vadeq 4242.40087890625 0.09074249402641961 0.033146796812830705
--- --- --- ---
StationName.bay754 0.027708787471055984 5.926748917277865e-07 2.1649475720215132e

In [38]:
model_set=aml_leaderboard_df['model_id']

mod_best=h2o.get_model(model_set[0])
mod_best

Out[55]: Model Details
=============
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_BestOfFamily_AutoML_20200612_004216

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.24617756721917858
RMSE: 0.49616284344878003
MAE: 0.14920699545240343
RMSLE: 0.07349810890166997
R^2: 0.9268891443645427
Mean Residual Deviance: 0.24617756721917858
Null degrees of freedom: 10021
Residual degrees of freedom: 10019
Null deviance: 33745.94852217975
Residual deviance: 2467.1915786706077
AIC: 14401.344656764826

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.47847185063434794
RMSE: 0.6917165970499392
MAE: 0.2992496882248374
RMSLE: 0.14071251719192907
R^2: 0.8434131329089226
Mean Residual Deviance: 0.47847185063434794
Null degrees of freedom: 24224
Residual degrees of freedom: 24222
Null deviance: 74025.41236472882
Residual deviance: 11590.98058161708
AIC: 50897.92185059927

In [39]:
mod_best._id

Out[30]: 'StackedEnsemble_BestOfFamily_AutoML_20200612_004216'

In [40]:
testing_frame = h2o.H2OFrame(testDF.toPandas())

predictedDF_automl = mod_best.predict(testing_frame)

perf = mod_best.model_performance(testing_frame)
perf

Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%
Test/Validation dataset column 'StationName' has levels not trained on: [bay170, bay233, bay243, bay244, bay325, bay579, bay595, bay615, bay635, bay655]
Out[54]: 
ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 0.2289448957438333
RMSE: 0.4784818656373858
MAE: 0.2808522061747738
RMSLE: 0.13180370623511828
R^2: 0.914524546789686
Mean Residual Deviance: 0.2289448957438333
Null degrees of freedom: 6095
Residual degrees of freedom: 6093
Null deviance: 16330.576260491764
Residual deviance: 1395.6480844544078
AIC: 8320.524693195197

In [41]:
# Using non ensable model for Variance importance
mod_best1=h2o.get_model(model_set[1])
mod_best1

Out[47]: Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_grid__1_AutoML_20200612_004216_model_13


Model Summary: 
 number_of_trees
-- -----------------
 51


ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.08432889058822349
RMSE: 0.29039437079293307
MAE: 0.11705731425061437
RMSLE: 0.060013769950949904
Mean Residual Deviance: 0.08432889058822349

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 0.5010054231970625
RMSE: 0.7078173657074701
MAE: 0.316772349769609
RMSLE: NaN
Mean Residual Deviance: 0.5010054231970625

Cross-Validation Metrics Summary: 
 mean sd cv_1_valid cv_2_valid cv_3_valid cv_4_valid cv_5_valid
---------------------- -------- ---------- ------------ ------------ ------------ ------------ ------------
mae 0.316772 0.00317831 0.319462 0.315005 0.313057 0.320654 0.315684
mean_residual_deviance 0.501005 0.274601 0.477308 0.350885 0.981371 0.327879 0.367584
mse 0.501005 0.274601 0.477308 0.350885 0.981371 0.327879 0.367584
r2 0.842041 0.0608288 0.838828 0.871324 0.738029 0.887206 0.874819
residual_deviance 0.501005 0.274601 0.477308 0.350885 0.981371 0.327879 0.367584
rmse 0.690553 0.173715 0.690875 0.592355 0.990642 0.572607 0.606287
rmsle 0.150458 0.00358092 nan 0.151691 0.146189 0.149319 0.154633

Scoring History: 
 timestamp duration number_of_trees training_rmse training_mae training_deviance
-- ------------------- ----------------- ----------------- --------------- -------------- -------------------
 2020-06-12 01:08:21 14 min 53.100 sec 0 2.22269 1.4136 4.94037
 2020-06-12 01:08:22 14 min 53.729 sec 5 0.845643 0.396743 0.715111
 2020-06-12 01:08:23 14 min 54.410 sec 10 0.608686 0.248764 0.370498
 2020-06-12 01:08:24 14 min 55.376 sec 15 0.515188 0.201671 0.265418
 2020-06-12 01:08:24 14 min 56.286 sec 20 0.460346 0.177569 0.211918
 2020-06-12 01:08:25 14 min 57.246 sec 25 0.418055 0.162916 0.17477
 2020-06-12 01:08:26 14 min 58.335 sec 30 0.387147 0.150626 0.149883
 2020-06-12 01:08:28 14 min 59.456 sec 35 0.358197 0.139768 0.128305
 2020-06-12 01:08:29 15 min 0.621 sec 40 0.335482 0.133688 0.112549
 2020-06-12 01:08:30 15 min 1.842 sec 45 0.315221 0.126582 0.0993645
 2020-06-12 01:08:31 15 min 3.143 sec 50 0.295465 0.119407 0.0872993
 2020-06-12 01:08:32 15 min 4.083 sec 51 0.290394 0.117057 0.0843289

Variable Importances: 
variable relative_importance scaled_importance percentage
------------------------- --------------------- --------------------- ----------------------
Longitude 46752.0859375 1.0 0.36528417218927267
Latitude 31911.30078125 0.6825642137959419 0.24932990380247236
HUC12 13435.775390625 0.2873834422829062 0.104976622815215
tp 9104.6708984375 0.1947436294202782 0.07113676546192081
GroupCode.vadeq 4242.40087890625 0.09074249402641961 0.033146796812830705
--- --- --- ---
StationName.bay754 0.027708787471055984 5.926748917277865e-07 2.1649475720215132e-07
StationName.mana5 0.02035534381866455 4.353889973139672e-07 1.5904070946414997e-07
StationName.bay497 0.019360080361366272 4.141008892575954e-07 1.5126450053530241e-07
StationName.8-col-40-laca 0.013500571250915527 2.88769388150159e-07 1.054828869040336e-07
StationName.bay442 0.010128090158104897 2.166339737577596e-07 7.913296177217583e-08

See the whole table with table.as_data_frame()

In [42]:
mod_best1.varimp(use_pandas=True)

,variable,relative_importance,scaled_importance,percentage
0,Longitude,46752.085938,1.000000e+00,3.652842e-01
1,Latitude,31911.300781,6.825642e-01,2.493299e-01
2,HUC12,13435.775391,2.873834e-01,1.049766e-01
3,tp,9104.670898,1.947436e-01,7.113677e-02
4,GroupCode.vadeq,4242.400879,9.074249e-02,3.314680e-02
5,GroupCode.padep,3408.214844,7.289974e-02,2.662912e-02
6,watertemp,2152.591797,4.604269e-02,1.681867e-02
7,StationName.cobr3,1948.132080,4.166942e-02,1.522118e-02
8,airtemp_narr,1777.897949,3.802821e-02,1.389110e-02
9,precip24_narr,1492.083374,3.191480e-02,1.165797e-02


In [43]:
mod_best1.varimp_plot()

## 4. Model comparison & Selection

Novel comparison of models

In [45]:
# Linear Regression
print("Linear Regression Metrics: ")
printEval(predictedDF_lr)
print(" ")

# Random Forest
print("Random Forest Metrics: ")
printEval(predictedDF_rf)
print(" ")

# GBT
print("GBT Metrics: ")
printEval(predictedDF_gbt)
print(" ")

# xgBoost
print("xgBoost Metrics: ")
print("RMSE: {}\nR2: {}\nMSE: {}\nMAE: {}\nVariance Explained: {}".format(r2, rmse, mse, mae, ve))
print(" ")

Linear Regression Metrics: 
RMSE: 1.3352019655186673
R2: 0.33305892314168517
MSE: 1.7827642887249122
MAE: 0.9148119756791776
Variance Explained: 1.037181236656476
 
Random Forest Metrics: 
RMSE: 0.7733119914935228
R2: 0.7762809173668949
MSE: 0.5980114361876783
MAE: 0.4493976915351382
Variance Explained: 1.8882213265937544
 
GBT Metrics: 
RMSE: 0.9569552642023226
R2: 0.6574083196959442
MSE: 0.915763377684537
MAE: 0.637145980034372
Variance Explained: 1.7340869158589196
 
xgBoost Metrics: 
RMSE: 0.7767041723450037
R2: 0.7955393090744154
MSE: 0.6328827922825981
MAE: 0.45335846370995736
Variance Explained: 0.7767049138628799

In [46]:
# AutoML
print("H2o AutoML Non - Ensamble Metrics: AutoML xgBoost ")
mod_best1

H2o AutoML Non - Ensamble Metrics: AutoML xgBoost 
Out[62]: Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_grid__1_AutoML_20200612_004216_model_13


Model Summary: 
 number_of_trees
-- -----------------
 51


ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.08432889058822349
RMSE: 0.29039437079293307
MAE: 0.11705731425061437
RMSLE: 0.060013769950949904
Mean Residual Deviance: 0.08432889058822349

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 0.5010054231970625
RMSE: 0.7078173657074701
MAE: 0.316772349769609
RMSLE: NaN
Mean Residual Deviance: 0.5010054231970625

Cross-Validation Metrics Summary: 
 mean sd cv_1_valid cv_2_valid cv_3_valid cv_4_valid cv_5_valid
---------------------- -------- ---------- ------------ ------------ ------------ ------------ ------------
mae 0.316772 0.00317831 0.319462 0.315005 0.313057 0.320654 0.315684
mean_residual_deviance 0.501005 0.274601 0.477308 0.350885 0.981371 0.327879 0.367584
mse 0.501005 0.274601 0.477308 0.350885 0.981371 0.327879 0.367584
r2 0.842041 0.0608288 0.838828 0.871324 0.738029 0.887206 0.874819
residual_deviance 0.501005 0.274601 0.477308 0.350885 0.981371 0.327879 0.367584
rmse 0.690553 0.173715 0.690875 0.592355 0.990642 0.572607 0.606287
rmsle 0.150458 0.00358092 nan 0.151691 0.146189 0.149319 0.154633

Scoring History: 
 timestamp duration number_of_trees training_rmse training_mae training_deviance
-- ------------------- ----------------- ----------------- --------------- -------------- -------------------
 2020-06-12 01:08:21 14 min 53.100 sec 0 2.22269 1.4136 4.94037
 2020-06-12 01:08:22 14 min 53.729 sec 5 0.845643 0.396743 0.715111
 2020-06-12 01:08:23 14 min 54.410 sec 10 0.608686 0.248764 0.370498
 2020-06-12 01:08:24 14 min 55.376 sec 15 0.515188 0.201671 0.265418
 2020-06-12 01:08:24 14 min 56.286 sec 20 0.460346 0.177569 0.211918
 2020-06-12 01:08:25 14 min 57.246 sec 25 0.418055 0.162916 0.17477
 2020-06-12 01:08:26 14 min 58.335 sec 30 0.387147 0.150626 0.149883
 2020-06-12 01:08:28 14 min 59.456 sec 35 0.358197 0.139768 0.128305
 2020-06-12 01:08:29 15 min 0.621 sec 40 0.335482 0.133688 0.112549
 2020-06-12 01:08:30 15 min 1.842 sec 45 0.315221 0.126582 0.0993645
 2020-06-12 01:08:31 15 min 3.143 sec 50 0.295465 0.119407 0.0872993
 2020-06-12 01:08:32 15 min 4.083 sec 51 0.290394 0.117057 0.0843289

Variable Importances: 
variable relative_importance scaled_importance percentage
------------------------- --------------------- --------------------- ----------------------
Longitude 46752.0859375 1.0 0.36528417218927267
Latitude 31911.30078125 0.6825642137959419 0.24932990380247236
HUC12 13435.775390625 0.2873834422829062 0.104976622815215
tp 9104.6708984375 0.1947436294202782 0.07113676546192081
GroupCode.vadeq 4242.40087890625 0.09074249402641961 0.033146796812830705
--- --- --- ---
StationName.bay754 0.027708787471055984 5.926748917277865e-07 2.1649475720215132e-07
StationName.mana5 0.02035534381866455 4.353889973139672e-07 1.5904070946414997e-07
StationName.bay497 0.019360080361366272 4.141008892575954e-07 1.5126450053530241e-07
StationName.8-col-40-laca 0.013500571250915527 2.88769388150159e-07 1.054828869040336e-07
StationName.bay442 0.010128090158104897 2.166339737577596e-07 7.913296177217583e-08

See the whole table with table.as_data_frame()

In [47]:
# AutoML
print("H2o AutoML Ensamble Metrics: ")
mod_best

H2o AutoML Ensamble Metrics: 
Out[64]: Model Details
=============
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_BestOfFamily_AutoML_20200612_004216

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.24617756721917858
RMSE: 0.49616284344878003
MAE: 0.14920699545240343
RMSLE: 0.07349810890166997
R^2: 0.9268891443645427
Mean Residual Deviance: 0.24617756721917858
Null degrees of freedom: 10021
Residual degrees of freedom: 10019
Null deviance: 33745.94852217975
Residual deviance: 2467.1915786706077
AIC: 14401.344656764826

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.47847185063434794
RMSE: 0.6917165970499392
MAE: 0.2992496882248374
RMSLE: 0.14071251719192907
R^2: 0.8434131329089226
Mean Residual Deviance: 0.47847185063434794
Null degrees of freedom: 24224
Residual degrees of freedom: 24222
Null deviance: 74025.41236472882
Residual deviance: 11590.98058161708
AIC: 50897.92185059927

In [48]:
perf

Out[65]: 
ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 0.2289448957438333
RMSE: 0.4784818656373858
MAE: 0.2808522061747738
RMSLE: 0.13180370623511828
R^2: 0.914524546789686
Mean Residual Deviance: 0.2289448957438333
Null degrees of freedom: 6095
Residual degrees of freedom: 6093
Null deviance: 16330.576260491764
Residual deviance: 1395.6480844544078
AIC: 8320.524693195197